In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import os
import json
import numpy as np
import torch
import xml.etree.ElementTree as ET
import sqlite3
import internal.utils.colmap as colmap

In [7]:
basic_path = "/mnt/Data/yanns/CityGaussian/data/lausanne_center"
image_dir_relative = "AerialPhotography"
def image_path_to_name(image_path):
    return image_path.split(":")[1][1:]

# Open XML
Looks like the new version has renamed to iTwin

In [8]:
# `fixed_pp` means the principle points are not adjusted
tree = ET.parse(os.path.join(basic_path, "cameras.xml"))
tree

In [9]:
root = tree.getroot()
root

<Element 'BlocksExchange' at 0x704ec4384720>

In [10]:
block = root.find("Block")
block

<Element 'Block' at 0x704ec43ac1d0>

In [11]:
images_subset = []

image_dir_txt = os.path.join(basic_path, "images.txt")
with open(image_dir_txt, "r") as f:
    lines = f.readlines()

for line in lines:
    if line.startswith("#"):
        continue
    parts = line.strip().split()
    if len(parts) < 7:
        continue
    image_path = parts[-1]
    images_subset.append(image_path)

In [12]:
cameras = []
image_paths = []
image_names = []
poses = []
centers = []
image_camera_ids = []
for photogroup in block.findall(".//Photogroup"):
    imageDimensions = photogroup.find("ImageDimensions")
    width = int(imageDimensions.find("Width").text)
    height = int(imageDimensions.find("Height").text)
    focal_length = float(photogroup.find("FocalLength").text)
    sensor_size = float(photogroup.find("SensorSize").text)

    focal_length_in_pixel = focal_length / sensor_size * width

    cameras.append({
        "width": width,
        "height": height,
        "focal_length": focal_length_in_pixel,
        "principal_point": (
            float(photogroup.find("PrincipalPoint/x").text), float(photogroup.find("PrincipalPoint/y").text)),
        "distortion": {i.tag: float(i.text) for i in photogroup.find("Distortion")} if photogroup.find("Distortion") is not None else {
            "K1": 0.0, "K2": 0.0, "P1": 0.0, "P2": 0.0
        },
    })
    camera_idx = len(cameras) - 1

    for photo in photogroup.findall("Photo"):
        rotation = list(photo.find("Pose/Rotation"))
        center = list(photo.find("Pose/Center"))
        if rotation[-1].text == "false":
            continue
        if center[-1].text == "false":
            continue

        path_split = photo.find("ImagePath").text.split("/")[-2:]
        path = "/".join(path_split)
        if path not in images_subset:
            continue
        image_paths.append(path)
        image_names.append(path_split[-1])
        poses.append([float(i.text) for i in rotation])
        centers.append([float(i.text) for i in center])
        image_camera_ids.append(camera_idx)
cameras

[{'width': 4000,
  'height': 3000,
  'focal_length': 28684.450198278606,
  'principal_point': (3062.4757710447493, 1863.5312040678798),
  'distortion': {'K1': -0.8503466409563408,
   'K2': 76.78396868259246,
   'K3': -5099.555347366658,
   'P1': -0.00766350587277634,
   'P2': 0.0038759429780537828}},
 {'width': 4160,
  'height': 6240,
  'focal_length': 1692.4572914949038,
  'principal_point': (2071.506927932276, 3111.5576327788403),
  'distortion': {'K1': -0.01498701112121934,
   'K2': 0.001471215222452104,
   'K3': 0.00014058471536766137,
   'P1': -0.0010365026590857566,
   'P2': -0.00013122397324184592}},
 {'width': 8192,
  'height': 5460,
  'focal_length': 8208.93986357127,
  'principal_point': (4066.61713573784, 2728.9156093120364),
  'distortion': {'K1': -0.05352835754755979,
   'K2': 0.07507916261378193,
   'K3': -0.2636898474337305,
   'P1': -0.00040454307115629927,
   'P2': 0.00041030987016786466}},
 {'width': 8192,
  'height': 5460,
  'focal_length': 8210.04464056159,
  'princ

In [13]:
poses[0], centers[0], image_paths[0]

([0.6273187084002719,
  0.778751939539123,
  0.004081023777044557,
  0.2631995277120081,
  -0.20708086440589185,
  -0.9422544901506621,
  -0.7329374098128646,
  0.59216799327637,
  -0.33487284308502313],
 [2538321.1497951504, 1152775.5691866004, 538.2550567952319],
 '10_Ancienne_Academie_RTK/DJI_20230420094603_0010_V.jpg')

Convert

In [14]:
pose_reshaped = torch.tensor(poses, dtype=torch.float64).reshape([-1, 3, 3])
pose_reshaped.shape, pose_reshaped[0]

(torch.Size([1887, 3, 3]),
 tensor([[ 0.6273,  0.7788,  0.0041],
         [ 0.2632, -0.2071, -0.9423],
         [-0.7329,  0.5922, -0.3349]], dtype=torch.float64))

In [15]:
c2w_rotations = torch.transpose(pose_reshaped, 1, 2)
(c2w_rotations[0] == pose_reshaped[0].T).all(), c2w_rotations[0]

(tensor(True),
 tensor([[ 0.6273,  0.2632, -0.7329],
         [ 0.7788, -0.2071,  0.5922],
         [ 0.0041, -0.9423, -0.3349]], dtype=torch.float64))

In [16]:
c2w = torch.concat([
    torch.concat([c2w_rotations, torch.tensor(centers, dtype=torch.float64)[..., None]], dim=-1),
    torch.tensor([0., 0., 0., 1.], dtype=torch.float64)[None, None, :].repeat(c2w_rotations.shape[0], 1, 1),
], dim=1)
c2w.shape, c2w[1], centers[1]

(torch.Size([1887, 4, 4]),
 tensor([[ 6.2765e-01,  2.6236e-01, -7.3296e-01,  2.5383e+06],
         [ 7.7849e-01, -2.0687e-01,  5.9259e-01,  1.1528e+06],
         [ 3.8437e-03, -9.4254e-01, -3.3408e-01,  5.3774e+02],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],
        dtype=torch.float64),
 [2538322.440174861, 1152777.2081968973, 537.7370503681381])

In [17]:
camera_centers = c2w[:, :3, 3]
camera_centers[0], centers[0]

(tensor([2.5383e+06, 1.1528e+06, 5.3826e+02], dtype=torch.float64),
 [2538321.1497951504, 1152775.5691866004, 538.2550567952319])

# Rescale and Translation

In [18]:
mean_center = torch.mean(camera_centers, dim=0)
mean_center

tensor([2.5383e+06, 1.1528e+06, 5.3967e+02], dtype=torch.float64)

In [19]:
camera_center_min = torch.min(camera_centers, dim=0).values
camera_center_max = torch.max(camera_centers, dim=0).values
camera_center_range = camera_center_max - camera_center_min
camera_center_range

tensor([75.1446, 72.3476, 39.7840], dtype=torch.float64)

In [20]:
mid_center = (camera_center_min + camera_center_max) * 0.5
mid_center

tensor([2.5384e+06, 1.1528e+06, 5.5046e+02], dtype=torch.float64)

In [21]:
max_range = 100.
scale = camera_center_range.max() / max_range
scale

tensor(0.7514, dtype=torch.float64)

In [22]:
c2w_rescaled_and_moved = torch.clone(c2w)
c2w_rescaled_and_moved[:, :3, 3] -= mid_center
c2w_rescaled_and_moved[:, :3, 3] /= scale
c2w[0], c2w_rescaled_and_moved[0]

(tensor([[ 6.2732e-01,  2.6320e-01, -7.3294e-01,  2.5383e+06],
         [ 7.7875e-01, -2.0708e-01,  5.9217e-01,  1.1528e+06],
         [ 4.0810e-03, -9.4225e-01, -3.3487e-01,  5.3826e+02],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],
        dtype=torch.float64),
 tensor([[ 6.2732e-01,  2.6320e-01, -7.3294e-01, -4.4776e+01],
         [ 7.7875e-01, -2.0708e-01,  5.9217e-01, -1.5581e+01],
         [ 4.0810e-03, -9.4225e-01, -3.3487e-01, -1.6236e+01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],
        dtype=torch.float64))

In [23]:
torch.save({
    "image_names": image_names,
    "cameras": cameras,
    "c2w": c2w,
    "image_camera_ids": image_camera_ids,
    "center": mid_center,
    "scale": scale,
}, os.path.join(basic_path, "parsed_from_xml.pt"))

# Select a few for preview in NeRFStudio

In [24]:
distance2center = torch.norm(camera_centers - mean_center[None, :], dim=-1)
distance2center

tensor([28.9990, 27.0974, 25.6044,  ..., 16.9749, 15.8031, 15.2447],
       dtype=torch.float64)

In [25]:
select_mask = distance2center < 128.
select_mask.sum()

tensor(1887)

In [26]:
selected_image_ids = select_mask.nonzero().squeeze(-1)
selected_image_ids

tensor([   0,    1,    2,  ..., 1884, 1885, 1886])

In [27]:
camera_list = []
for idx, pose in enumerate(c2w[select_mask]):
    camera_list.append({
        "id": idx,
        "img_name": "{:06d}".format(idx),
        "width": 1920,
        "height": 1080,
        "position": (pose[:3, 3] * 0.01).tolist(),
        "rotation": pose[:3, :3].tolist(),
        "fx": 1600,
        "fy": 1600,
        "color": [255, 0, 0],
    })
with open(os.path.join(os.path.expanduser("~/data/image_set/dbl"), "preview.json"), "w") as f:
    json.dump(camera_list, f)
os.path.join(os.path.expanduser("~/data/image_set/dbl"), "preview.json")

FileNotFoundError: [Errno 2] No such file or directory: '/home/yanns/data/image_set/dbl/preview.json'

In [28]:
transforms = {
    "aabb_scale": 16,
}

frames = []
for idx in selected_image_ids.tolist():
    camera_id = image_camera_ids[idx]
    file_path = os.path.join(image_dir_relative, image_names[idx])
    camera = cameras[camera_id]
    transform_matrix = torch.clone(c2w[idx])
    transform_matrix[:, 1:3] *= -1
    frames.append({
        "file_path": file_path,
        "camera_model": "OPENCV",
        "fl_x": camera["focal_length"],
        "fl_y": camera["focal_length"],
        "k1": camera["distortion"]["K1"],
        "k2": camera["distortion"]["K2"],
        "p1": camera["distortion"]["P1"],
        "p2": camera["distortion"]["P2"],
        "cx": camera["width"] // 2,
        "cy": camera["height"] // 2,
        "w": camera["width"],
        "h": camera["height"],
        "transform_matrix": transform_matrix.tolist(),
    })

transforms["frames"] = frames

transforms_json_path = os.path.join(os.path.expanduser("~/data/image_set/dbl"), "transforms.json")
with open(transforms_json_path, "w") as f:
    json.dump(transforms, f, indent=2)
transforms_json_path

FileNotFoundError: [Errno 2] No such file or directory: '/home/yanns/data/image_set/dbl/transforms.json'

# Colmap

In [29]:
colmap_output_path = os.path.join(basic_path, "colmap")
image_dir_relative = "images"  # relative path to the images directory
colmap_image_path = os.path.join(basic_path, image_dir_relative)


In [39]:
colmap_db_path = os.path.join(colmap_output_path, "colmap.db")
#assert os.path.exists(colmap_db_path) is False
print(" \\\n    ".join([
    "colmap",
    "feature_extractor",
    "--database_path=" + colmap_db_path,
    "--image_path=" + colmap_image_path,
    "--ImageReader.camera_model=OPENCV",
]))

colmap \
    feature_extractor \
    --database_path=/mnt/Data/yanns/CityGaussian/data/lausanne_center/colmap/colmap.db \
    --image_path=/mnt/Data/yanns/CityGaussian/data/lausanne_center/images \
    --ImageReader.camera_model=OPENCV


create a sparse model from known poses

In [40]:
sparse_manually_model_dir = os.path.join(colmap_output_path, "sparse_manually")

In [44]:
#assert os.path.exists(sparse_manually_model_dir) is False
#assert os.path.exists(colmap_db_path + "-shm") is False, "{} is opened by another process".format(colmap_db_path)

colmap_db = sqlite3.connect(colmap_db_path)

def array_to_blob(array):
    return array.tostring()


def select_image(image_name: str):
    cur = colmap_db.cursor()
    try:
        return cur.execute("SELECT image_id, camera_id FROM images WHERE name = ?", [image_name]).fetchone()
    except sqlite3.OperationalError as e:
        print("No image found with name:", image_name)
        return (None, None)
    finally:
        cur.close()


def set_image_camera_id(image_id: int, camera_id: int):
    cur = colmap_db.cursor()
    try:
        cur.execute("UPDATE images SET camera_id = ? WHERE image_id = ?", [camera_id, image_id])
        colmap_db.commit()
    finally:
        cur.close()


def update_camera_params(camera_id: int, params: np.ndarray):
    cur = colmap_db.cursor()
    try:
        cur.execute("UPDATE cameras SET params = ? WHERE camera_id = ?", [
            array_to_blob(params),
            camera_id,
        ])
        colmap_db.commit()
    finally:
        cur.close()

def remove_from_db(image_name: str):
    cur = colmap_db.cursor()
    try:
        cur.execute("DELETE FROM images WHERE name = ?", [image_name])
        colmap_db.commit()
    except sqlite3.OperationalError as e:
        print("Error deleting image:", e)
    finally:
        cur.close()


def delete_unused_cameras():
    cur = colmap_db.cursor()
    try:
        cur.execute("DELETE FROM cameras WHERE camera_id NOT IN (SELECT camera_id FROM images)")
        colmap_db.commit()
    finally:
        cur.close()

In [45]:
w2cs = torch.linalg.inv(c2w_rescaled_and_moved)
w2cs[0], c2w_rescaled_and_moved[0]

(tensor([[ 6.2732e-01,  7.7875e-01,  4.0810e-03,  4.0289e+01],
         [ 2.6320e-01, -2.0708e-01, -9.4225e-01, -6.7397e+00],
         [-7.3294e-01,  5.9217e-01, -3.3487e-01, -2.9029e+01],
         [ 6.0286e-20, -3.3706e-18, -5.4978e-19,  1.0000e+00]],
        dtype=torch.float64),
 tensor([[ 6.2732e-01,  2.6320e-01, -7.3294e-01, -4.4776e+01],
         [ 7.7875e-01, -2.0708e-01,  5.9217e-01, -1.5581e+01],
         [ 4.0810e-03, -9.4225e-01, -3.3487e-01, -1.6236e+01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],
        dtype=torch.float64))

In [46]:
colmap_cameras = {}
colmap_images = {}
context_camera_idx_to_colmap_camera_idx = {}

i = 0
for idx in range(c2w.shape[0]):
    image_name = image_paths[idx]
    if not select_image(image_name) :
        remove_from_db(image_name)
        continue
    colmap_image_idx, colmap_camera_idx = select_image(image_name)
    context_camera_idx = image_camera_ids[idx]
    colmap_camera_idx = context_camera_idx_to_colmap_camera_idx.setdefault(context_camera_idx, colmap_camera_idx)
    set_image_camera_id(colmap_image_idx, colmap_camera_idx)

    w2c = w2cs[idx]

    colmap_images[colmap_image_idx] = colmap.Image(
        id=colmap_image_idx,
        qvec=colmap.rotmat2qvec(w2c[:3, :3].numpy()),
        tvec=w2c[:3, 3].numpy(),
        camera_id=colmap_camera_idx,
        name=image_name,
        xys=np.array([], dtype=np.float64),
        point3D_ids=np.asarray([], dtype=np.int64),
    )

    if colmap_camera_idx not in colmap_cameras:
        camera = cameras[context_camera_idx]
        # [fx, fy, cx, cy, k1, k2, p1, p2]
        camera_params = torch.tensor([
            camera["focal_length"],
            camera["focal_length"],
            camera["width"] // 2,
            camera["height"] // 2,
            camera["distortion"]["K1"],
            camera["distortion"]["K2"],
            camera["distortion"]["P1"],
            camera["distortion"]["P2"],
        ], dtype=torch.float64)
        update_camera_params(colmap_camera_idx, camera_params.numpy())
        colmap_cameras[colmap_camera_idx] = colmap.Camera(
            id=colmap_camera_idx,
            model="OPENCV",
            width=camera["width"],
            height=camera["height"],
            params=camera_params.numpy(),
        )
delete_unused_cameras()
#colmap_db.close()

/tmp/ipykernel_442454/2486671127.py:7: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  return array.tostring()


In [49]:
#see all camera ids
np.unique(list([i.camera_id for i in colmap_images.values()])).tolist()

[1, 2, 3]

In [50]:
#os.makedirs(sparse_manually_model_dir)
colmap.write_images_binary(colmap_images, os.path.join(sparse_manually_model_dir, "images.bin"))
colmap.write_cameras_binary(colmap_cameras, os.path.join(sparse_manually_model_dir, "cameras.bin"))
colmap.write_points3D_binary({}, os.path.join(sparse_manually_model_dir, "points3D.bin"))

In [51]:
colmap.read_cameras_binary(os.path.join(sparse_manually_model_dir, "cameras.bin"))

{1: Camera(id=1, model='OPENCV', width=5280, height=3956, params=array([ 3.77042959e+03,  3.77042959e+03,  2.64000000e+03,  1.97800000e+03,
        -1.93445480e-01,  2.14433841e-01, -2.65226290e-04,  5.05558491e-05])),
 2: Camera(id=2, model='OPENCV', width=6240, height=4160, params=array([ 2.52109519e+03,  2.52109519e+03,  3.12000000e+03,  2.08000000e+03,
         8.53234133e-03, -2.78956282e-02,  1.55481132e-04, -1.25667945e-04])),
 3: Camera(id=3, model='OPENCV', width=5184, height=3888, params=array([2289.29613371, 2289.29613371, 2592.        , 1944.        ,
           0.        ,    0.        ,    0.        ,    0.        ]))}

feature matcher

In [54]:
print(" \\\n    ".join([
    "colmap",
    "vocab_tree_matcher",
    "--database_path=" + colmap_db_path,
    "--VocabTreeMatching.vocab_tree_path=" + "data/lausanne/vocab_tree_faiss_flickr100K_words256K.bin",
]))

colmap \
    vocab_tree_matcher \
    --database_path=/mnt/Data/yanns/CityGaussian/data/lausanne_center/colmap/colmap.db \
    --VocabTreeMatching.vocab_tree_path=data/lausanne/vocab_tree_faiss_flickr100K_words256K.bin


point triangulator

In [55]:
sparse_dir_triangulated = os.path.join(colmap_output_path, "sparse")
os.makedirs(sparse_dir_triangulated, exist_ok=True)
print(" \\\n    ".join([
        "colmap",
        "point_triangulator",
        "--database_path=" + colmap_db_path,
        "--image_path=" + colmap_image_path,
        "--input_path=" + sparse_manually_model_dir,
        "--output_path=" + sparse_dir_triangulated,
]))

colmap \
    point_triangulator \
    --database_path=/mnt/Data/yanns/CityGaussian/data/lausanne_center/colmap/colmap.db \
    --image_path=/mnt/Data/yanns/CityGaussian/data/lausanne_center/images \
    --input_path=/mnt/Data/yanns/CityGaussian/data/lausanne_center/colmap/sparse_manually \
    --output_path=/mnt/Data/yanns/CityGaussian/data/lausanne_center/colmap/sparse


In [61]:
bundle_output_path = os.path.join(colmap_output_path, "bundle_adjusted")
print(" \\\n    ".join([
    "colmap ",
    "bundle_adjuster ",
    "--input_path " + sparse_dir_triangulated,
    "--output_path " + bundle_output_path
]))

colmap  \
    bundle_adjuster  \
    --input_path /mnt/Data/yanns/CityGaussian/data/lausanne_center/colmap/sparse \
    --output_path /mnt/Data/yanns/CityGaussian/data/lausanne_center/colmap/bundle_adjusted
